In [1]:
import pandas
import time
import argparse
import def_function
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import numpy

In [2]:
__version__ = "V1.2(Editor) 2023-08-05"

In [3]:
# 前置参数-debug
ATSS_APA_cutoff = 3
log2FC_cutoff = 1.5
plot_x_maxnum = 5
absolute_path = False
file_path = "F:/OneDrive/Master/Project/trans/data/"
input_sample_info_filename = "0000_sample_info.tsv"
input_df_filename = "0001_total_info.tsv"
input_TSS_TES_filename = "0002_result_check.tsv"
output_length_counts_plot_filename = "0004_length_counts.png"
output_check_df_filename = "0004_check_df.xlsx"
output_transcript_gene_plot_path = "0004_plot/"
output_relative_expression_filename = "0004_relative_expression.tsv"

In [4]:
# 前置参数
parser = argparse.ArgumentParser()
parser.add_argument("--ATSS_APA_cutoff", dest="ATSS_APA_cutoff", required=False, type=int, default=3, help="=3,\t the cutoff value of isoform of ATSS-APA gene")
parser.add_argument("--log2FC_cutoff", dest="log2FC_cutoff", required=False, type=float, default=1.5, help="=1.5,\t the cutoff value of log2FC")
parser.add_argument("--plot_x_maxnum", dest="plot_x_maxnum", required=False, type=int, default=5, help="=5,\t the maxnum in xaxis")
parser.add_argument("--absolute_path", dest="absolute_path", required=False, action="store_true", help="use the absolute path")
parser.add_argument("--file_path", dest="file_path", required=False, type=str, default="./", help="=the data directory path")
parser.add_argument("--input_sample_info_filename", dest="input_sample_info_filename", required=False, type=str, default="0000_sample_info.tsv", help="=0000_sample_info.tsv,\t the metadata of samples")
parser.add_argument("--input_df_filename", dest="input_df_filename", required=False, type=str, default="0001_total_info.tsv", help="=0001_total_info.tsv,\t the output file of 0001.py")
parser.add_argument("--input_TSS_TES_filename", dest="input_TSS_TES_filename", required=False, type=str, default="0002_result_check.tsv", help="=0002_result_check.tsv,\t the output file of 0002.py")
parser.add_argument("--output_length_counts_plot_filename", dest="output_length_counts_plot_filename", required=False, type=str, default="0004_length_counts.png", help="=0004_length_counts.png,\t to confirm rep is not related to length of transcript")
parser.add_argument("--output_check_df_filename", dest="output_check_df_filename", required=False, type=str, default="0004_check_df.xlsx", help="=0004_check_df.xlsx,\t the output file for checking")
parser.add_argument("--output_transcript_gene_plot_path", dest="output_transcript_gene_plot_path", required=False, type=str, default="0004_plot/", help="=0004_plot/,\t the path of output plot")
parser.add_argument("--output_relative_expression_filename", dest="output_relative_expression_filename", required=False, type=str, default="0004_relative_expression.tsv", help="=0004_relative_expression.tsv,\t the output filename of cell_line expression")

args = parser.parse_args()
ATSS_APA_cutoff = args.ATSS_APA_cutoff
log2FC_cutoff = args.log2FC_cutoff
plot_x_maxnum = args.plot_x_maxnum
absolute_path = args.absolute_path
file_path = args.file_path
input_sample_info_filename = args.input_sample_info_filename
input_df_filename = args.input_df_filename
input_TSS_TES_filename = args.input_TSS_TES_filename
output_length_counts_plot_filename = args.output_length_counts_plot_filename
output_check_df_filename = args.output_check_df_filename
output_transcript_gene_plot_path = args.output_transcript_gene_plot_path
output_relative_expression_filename = args.output_relative_expression_filename

usage: ipykernel_launcher.py [-h] [--log2FC_cutoff LOG2FC_CUTOFF]
                             [--plot_x_maxnum PLOT_X_MAXNUM] [--absolute_path]
                             [--file_path FILE_PATH]
                             [--input_sample_info_filename INPUT_SAMPLE_INFO_FILENAME]
                             [--input_df_filename INPUT_DF_FILENAME]
                             [--input_TSS_TES_filename INPUT_TSS_TES_FILENAME]
                             [--output_length_counts_plot_filename OUTPUT_LENGTH_COUNTS_PLOT_FILENAME]
                             [--output_check_df_filename OUTPUT_CHECK_DF_FILENAME]
                             [--output_transcript_gene_plot_path OUTPUT_TRANSCRIPT_GENE_PLOT_PATH]
                             [--output_relative_expression_filename OUTPUT_RELATIVE_EXPRESSION_FILENAME]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"bcd69888-ac5f

SystemExit: 2

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# 补充路径
if absolute_path is False:
    input_sample_info_filename = "{}{}".format(file_path, input_sample_info_filename)
    input_df_filename = "{}{}".format(file_path, input_df_filename)
    input_TSS_TES_filename = "{}{}".format(file_path, input_TSS_TES_filename)
    output_length_counts_plot_filename = "{}other/{}".format(file_path, output_length_counts_plot_filename)
    output_check_df_filename = "{}{}".format(file_path, output_check_df_filename)
    output_transcript_gene_plot_path = "{}{}".format(file_path, output_transcript_gene_plot_path)
    output_relative_expression_filename = "{}{}".format(file_path, output_relative_expression_filename)

In [ ]:
# 打印参数
print('\n')
print("[Date]{}".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))))
print("[Script]{}".format(__file__))
print("[Version]{}".format(__version__))
print("[Version]def_function: {}".format(def_function.__version__))
print("[Parament]ATSS_APA_cutoff: {}".format(ATSS_APA_cutoff))
print("[Parament]log2FC_cutoff: {}".format(log2FC_cutoff))
print("[Parament]plot_x_maxnum: {}".format(plot_x_maxnum))
print("[Parament]file_path: {}".format(file_path))
print("[Parament]input_sample_info_filename: {}".format(input_sample_info_filename))
print("[Parament]input_df_filename: {}".format(input_df_filename))
print("[Parament]input_TSS_TES_filename: {}".format(input_TSS_TES_filename))
print("[Parament]output_length_counts_plot_filename: {}".format(output_length_counts_plot_filename))
print("[Parament]output_check_df_filename: {}".format(output_check_df_filename))
print("[Parament]output_transcript_gene_plot_path: {}".format(output_transcript_gene_plot_path))
print("[Parament]output_relative_expression_filename: {}".format(output_relative_expression_filename))
print('\n')

In [5]:
@def_function.log
def output_length_counts_check_plot(sample_info, total_info, output_filename, tab_level=0):
    """
    input:
        sample_info, df
        total_info, df
        output_filename, str
    change:
        get the plot which xaxis is length of transcript and yaxis is counts of transcript
    output:
        None
    """
    sample_info = sample_info
    total_info = total_info
    output_filename = output_filename

    fig_info_dict = {}
    for sample in sample_info["GEO_accession"]:
        temp_df = total_info[[sample, "transcript_start", "transcript_end"]]
        temp_df = temp_df.loc[temp_df[sample]>0, :]  # get the rows of count>0
        temp_df["length"] = temp_df["transcript_end"] - temp_df["transcript_start"]
        temp_df = temp_df[[sample, "length"]]

        fig_info = go.Scatter(x=temp_df["length"],
                              y=temp_df[sample],
                              name=sample,
                              mode="markers",
                              marker={"size": 2.5})
        
        fig_info_dict[sample] = fig_info

    #rows = math.ceil(math.sqrt(sample_info.shape[0]))
    #cols = math.ceil(sample_info.shape[0]/rows)
    fig = make_subplots(rows=1, cols=1,
                        shared_xaxes=True, shared_yaxes=True)
    for sample, fig_info in fig_info_dict.items():
        fig.add_trace(fig_info)
    
    layout = {"title": {"font": {"family": "Arial",
                                 "size": 12,
                                 "color": "black"},
                        },
              "font": {"family": "Arial",
                       "size": 10,
                       "color": "black"},
              "margin": {'l':20, 'r':20, 't':20, 'b':20},
              "xaxis": {"title": {"text": "transcripts length",
                                  "font": {"family": "Arial",
                                           "color": "black",
                                           "size": 12},
                                  "standoff": 0.5},
                        "showline": True,
                        "linecolor": "black",
                        },
              "yaxis": {"title": {"text": "transcripts counts",
                                  "font": {"family": "Arial",
                                           "color": "black",
                                           "size": 12},
                                  "standoff": 0.5},
                        "showline": True,
                        "linecolor": "black",
                        },
              "paper_bgcolor": "white",
              "plot_bgcolor": "white",
              "showlegend": False
              }

    fig.update_layout(layout)

    fig.write_image(output_length_counts_plot_filename, scale=4)

    return None


@def_function.log
def compute_expression_for_each_cell_line(total_info, sample_info,
                                          cell_line_sample_dict, tab_level=0):
    """
    input:
        total_info, pandas.DataFrame, ...
        sample_info, pandas.DataFrame, ...
        cell_line_sample_dict, dict, {cell_line1: [sample1, sample2, ...], ...}
    change:
        一个transcript在一个样本中的相对表达量 = 该transcript的rep值 / 该样本所有transcript的rep值的和 * 10**6
        该transcript在相同疾病类型中的不同样本中的平均表达量= 该transcript在一种类型疾病中所有样本的相对表达量的加和 / 该类型疾病的样本数
    output:
        total_info, pandas.DataFrame, expression in cell_line added
    """
    total_info = total_info
    sample_info = sample_info
    cell_line_sample_dict = cell_line_sample_dict

    # 计算transcript在单个样本中的相对表达量
    for sample in sample_info["GEO_accession"]:
        sample_sum = total_info[sample].sum()
        total_info["{}_expression_sample".format(sample)] = total_info[sample].map(lambda x: x/sample_sum * 10**6)
    # 计算transcript在同种疾病类型的不同样本中的平均相对表达量
    for cell_line, sample_list in cell_line_sample_dict.items():
        column_name = "{}_expression".format(cell_line)
        sample_list_expression = ["{}_expression_sample".format(i) for i in sample_list]
        total_info[column_name] = total_info[sample_list_expression].apply(lambda x: x.sum(), axis=1)
        total_info[column_name] = total_info[column_name] / len(sample_list)
    # drop <sample>_expression_sample column
    drop_column = ["{}_expression_sample".format(i) for i in sample_info["GEO_accession"]]
    total_info = total_info.drop(labels=drop_column, axis=1)

    return total_info


@def_function.log
def get_ATSS_APA_gene_in_cell_line(df, gene_biotype, cell_line_1, cell_line_2, tab_level=0):
    """
    input:
        df, pandas.DataFrame, the output file of 0002.py
        gene_biotype, str, ...
        transcript_biotype, str, ...
        cell_line_1, str, 注意尽量以下划线代替空格
        cell_line_2, str, 注意尽量以下划线代替空格
    change:
        ...
    output:
        # result, dict, {<gene_id>: <transcript_counts>, ...}
        result, set, 指定gene_biotype的ATSS-APA基因id
    """
    df = df
    gene_biotype = gene_biotype
    cell_line_1 = cell_line_1
    cell_line_2 = cell_line_2

    cell_line_1 = cell_line_1.replace(' ', '_')
    cell_line_2 = cell_line_2.replace(' ', '_')
    df["cell_line"] = df["cell_line"].map(lambda x: x.replace(' ', '_'))

    # 根据所有样本的数据筛选指定gene_biotype的ATSS-APA基因
    temp_df = df.loc[df["cell_line"]=="all", :]
    if gene_biotype != "all":
        temp_df = temp_df.loc[temp_df["gene_biotype"]==gene_biotype, :]  # retain specified gene_biotype
    temp_df = temp_df.query("start_counts >=2 & end_counts >=2")  # retain ATSS-PAS gene
    ATSS_APA_gene = set(temp_df["gene_id"].to_list())
    

    temp_df = df.loc[df["cell_line"]==cell_line_1, :]
    if gene_biotype != "all":
        temp_df = temp_df.loc[temp_df["gene_biotype"]==gene_biotype, :]  # retain specified gene_biotype
    cell_line_1_gene = temp_df["gene_id"].to_list()

    temp_df = df.loc[df["cell_line"]==cell_line_2, :]
    if gene_biotype != "all":
        temp_df = temp_df.loc[temp_df["gene_biotype"]==gene_biotype, :]  # retain specified gene_biotype
    cell_line_2_gene = temp_df["gene_id"].to_list()

    # cell_line_ATSS_PAS_gene, set, 在两种类型疾病中都存在表达的指定gene_biotype的ATSS-APA基因
    cell_line_gene = set(cell_line_1_gene+cell_line_2_gene)
    cell_line_ATSS_APA_gene = ATSS_APA_gene.intersection(cell_line_gene)

    """# get result dict
    result = {}
    result_index = list(cell_line_ATSS_APA_gene)
    df = df.loc[df["cell_line"]=="all", :]
    df = df.set_index("gene_id")
    df = df.loc[result_index, :]
    for gene_id in df.index:
        result[gene_id] = df.at[gene_id, "transcript_counts"]"""
    
    result = cell_line_ATSS_APA_gene

    return result


@def_function.log
def compute_counts_of_gene(significant_gene_transcript, max_num=5, tab_level=0):
    """
    input:
        significant_gene_transcript, dict, {<gene_id>: <the counts of significant transcript of gene>, ...}
        max_num, int, the max num of compute
    change:
        根据significant_gene_transcript, 统计具有不同显著表达差异的transcript数目的gene数
        如果一个基因所具有的显著表达差异transcript的数目超过了max_num, 就认为该gene具有max_num个显著表达差异transcript
    output:
        counts_num, dict, {<counts of significant transcript>: <counts of gene>}
    """
    max_num = 5
    counts_num = {num: 0 for num in range(1,max_num+1)}
    for gene_id, counts in significant_gene_transcript.items():
        if counts <= max_num:
            counts_num[counts] = counts_num.get(counts) + 1
        else:
            counts_num[max_num] = counts_num.get(max_num) +1
    
    return counts_num


@def_function.log
def get_counts_of_transcript_of_gene(isoform_info_df, total_info,
                                     cell_line_1, cell_line_2,
                                     gene_biotype, transcript_biotype=["all"],
                                     log2FC_cutoff=1.5, max_num=5,
                                     tab_level=0):
    """
    input:
        isoform_info_df, pandas.DataFrame, the output of 0002.py
        total_info, pandas.DataFrame, ...
        gene_biotype, str, ...
        transcript_biotype, list=["all"], the transcript will be retained. If "all" in this list, the transcript won't be filtered according to the transcript_biotype.
        log2FC_cutoff, float=1.5, the cutoff value of log2FC
        max_num, int=5, the maxnum of significant expressed transcript in one gene, 如果值大于该阈值则会将值修改为该阈值
    change:
    output:
        dict={"check_df": check_df, "result": result}, check_df is df and result is dict
    """
    isoform_info_df = isoform_info_df
    total_info = total_info
    gene_biotype = gene_biotype
    transcript_biotype = transcript_biotype
    cell_line_1 = cell_line_1
    cell_line_2 = cell_line_2
    log2FC_cutoff = log2FC_cutoff
    max_num = max_num


    ATSS_APA_gene_set = get_ATSS_APA_gene_in_cell_line(df=isoform_info_df,
                                                       gene_biotype=gene_biotype,
                                                       cell_line_1=cell_line_1,
                                                       cell_line_2=cell_line_2,
                                                       tab_level=tab_level+1)
    print("{}[Result]the counts of gene about ATSS-APA and {} gene: {}".format('\t'*(tab_level+1), gene_biotype, len(ATSS_APA_gene_set)))

    # gene and transcript filter
    if gene_biotype != "all":
        check_df = total_info.loc[total_info["gene_biotype"]==gene_biotype, :]  # specify gene_biotype
    else:
        check_df = total_info
    if "all" not in transcript_biotype:
        check_df = check_df.loc[check_df["transcript_biotype"].isin(transcript_biotype), :]  # specify transcript_biotype
    check_df = check_df.loc[check_df["gene_id"].isin(ATSS_APA_gene_set), :]  # specify gene_biotype and ATSS-PAS

    # change log format
    if len(transcript_biotype) <= 5:
        print("{}[Result]the counts of gene about ATSS-APA and {} gene and {} transcript: {}".format('\t'*(tab_level+1),
                                                                                                    gene_biotype,
                                                                                                    transcript_biotype,
                                                                                                    len(set(check_df["gene_id"]))))
        print("{}[Result]the counts of transcript about ATSS-APA and {} gene and {} transcript: {}".format('\t'*(tab_level+1),
                                                                                                        gene_biotype,
                                                                                                        transcript_biotype,
                                                                                                        check_df.shape[0]))
    else:
        print("{}[Result]the counts of gene about ATSS-APA and {} gene and <transcript_biotype> transcript: {}".format('\t'*(tab_level+1),
                                                                                                    gene_biotype,
                                                                                                    len(set(check_df["gene_id"]))))
        print("{}[Result]the counts of transcript about ATSS-APA and {} gene and <transcript_biotyoe> transcript: {}".format('\t'*(tab_level+1),
                                                                                                        gene_biotype,
                                                                                                        check_df.shape[0]))
    
    # 计算log2FC, 计算完成后，值有两种类型，一种是常规的log2FC值，另一种是numpy.nan, numpy.nan代表着该transcript在两种类型疾病中的一种中的相对表达量为0
    check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)] = check_df["{}_expression".format(cell_line_1)] / check_df["{}_expression".format(cell_line_2)]
    check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)] = check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)].map(lambda x: numpy.nan if numpy.isinf(x) or x==0 else x)
    check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)] = check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)].map(lambda x: math.log(x, 2))

    # 筛选出在两种疾病类型中具有显著表达差异的transcript
    ## numpy.nan
    temp_df = map(lambda x: numpy.isnan(x), check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)])
    temp_df = list(map(lambda x: numpy.isnan(x), check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)]))
    temp_df = check_df.loc[temp_df, :]
    ## log2FC >= cutoff
    check_df = check_df.loc[abs(
                                check_df["log2FC({}/{})".format(cell_line_1, cell_line_2)]
                                ) >= log2FC_cutoff, :]
    ## concat
    check_df = pandas.concat([check_df, temp_df], axis=0, ignore_index=True)
    if len(transcript_biotype) <= 5:
        print("{}[Result]the counts of gene showing significant expression in {} and {} about ATSS-APA and {} gene and {} transcript: {}".format('\t'*(tab_level+1),
                                                                    cell_line_1,
                                                                    cell_line_2,
                                                                    gene_biotype,
                                                                    transcript_biotype,
                                                                    len(set(check_df["gene_id"]))))
        print("{}[Result]the counts of transcript showing significant expression in {} and {} about ATSS-APA and {} gene and {} transcript: {}".format('\t'*(tab_level+1),
                                                                    cell_line_1,
                                                                    cell_line_2,
                                                                    gene_biotype,
                                                                    transcript_biotype,
                                                                    check_df.shape[0]))
    else:
        print("{}[Result]the counts of gene showing significant expression in {} and {} about ATSS-APA and {} gene and <transcript_biotype> transcript: {}".format('\t'*(tab_level+1),
                                                                                       cell_line_1,
                                                                                       cell_line_2,
                                                                                       gene_biotype,
                                                                                       len(set(check_df["gene_id"]))))
        print("{}[Result]the counts of transcript showing significant expression in {} and {} about ATSS-APA and {} gene and <transcript_biotype> transcript: {}".format('\t'*(tab_level+1),
                                                                                       cell_line_1,
                                                                                       cell_line_2,
                                                                                       gene_biotype,
                                                                                       check_df.shape[0]))

    significant_gene_transcript = {}  # {<gene_id>: <the num of significant transcript of gene>}
    check_df_group = check_df.groupby("gene_id")
    for gene_id in check_df_group.groups.keys():
        temp_df = check_df_group.get_group(gene_id)
        transcript_num = temp_df.shape[0]
        significant_gene_transcript[gene_id] = transcript_num

    result = compute_counts_of_gene(significant_gene_transcript,
                                    max_num=max_num,
                                    tab_level=tab_level+1)
    result[0] = len(ATSS_APA_gene_set) - len(set(check_df["gene_id"]))

    return {"check_df": check_df, "result": result}


# output file
@def_function.log
def output_check_df(df_dict, filename, tab_level=0):
    """
    input:
        df, dict, {<df_name>: <df>, ...}
    change:
        output all df to .xlsx file
    output:
        None
    """
    df_dict = df_dict
    filename = filename

    num = 0
    with pandas.ExcelWriter(filename, mode='w') as file:
        for key, df in df_dict.items():
            print("{}[Result]sheet_name should be: {}".format('\t'*(tab_level+1), key))
            key = key.split('@')

            num += 1
            key[2] = str(num)

            num += 1
            key[3] = str(num)

            key = '_'.join(key)
            print("{}[Result]sheet_name is: {}".format('\t'*(tab_level+1), key))

            df.to_excel(file, sheet_name=key, index=None)
    
    return None


@def_function.log
def make_transcript_gene_plot(df, cell_line_1, cell_line_2, gene_biotype, transcript_biotype, tab_level=0):
        plot_info_df = df
        cell_line_1 = cell_line_1
        cell_line_2 = cell_line_2
        gene_biotype = gene_biotype
        transcript_biotype = transcript_biotype

        temp_df = plot_info_df.query("cell_line_1=='{}' & cell_line_2=='{}' & gene_biotype=='{}' & transcript_biotype==\"{}\"".format(cell_line_1, cell_line_2, gene_biotype, transcript_biotype)).copy()
        temp_df = temp_df.drop(columns=["cell_line_1", "cell_line_2", "gene_biotype", "transcript_biotype"])
        temp_df["index"] = 'y'
        temp_df = temp_df.set_index("index")
        temp_df = temp_df.T
        temp_df['x'] = temp_df.index
        temp_num0 = temp_df.at[0, 'y']
        temp_df.at[0, 'y'] = 0

        temp_plot = go.Figure(go.Bar(x=temp_df['x'], y=temp_df['y'],
                                width=0.9,
                                text=["{}".format(temp_num0)],
                                textposition="outside",
                                textfont={"family": "Arial", "color": "black", "size": 12},
                                textangle=-90,
                                marker={"color": "purple",
                                        "line": {"color": "black",
                                                "width": 0.5},
                                        }))
        layout = {"width": 250, "height": 400,
                "title": {"text": "{}/{}".format(temp_df['y'][1:].sum(), temp_df['y'].sum()+temp_num0),
                        "font": {"family": "Arial",
                                "color": "black",
                                "size": 12},
                        'x': 0.8,
                        },
                "margin": {'l':15, 'r':15, 't':25, 'b':15}, 
                "barmode": "stack",
                "font": {"family": "Arial",
                        "color": "black",
                        "size": 10},
                "xaxis": {"title": {"text": "Per gene, 5'-3' isoforms <br> with a significant 5'-3' link",
                                "font": {"family": "Arial",
                                        "color": "black",
                                        "size": 12},
                                "standoff": 0,
                                },
                        "linecolor": "black",
                        "dtick": 1,
                        "ticksuffix": '+',
                        "showticksuffix": "last",
                        },
                "yaxis": {"title": {"text": "Number of Genes",
                                "font": {"family": "Arial",
                                        "color": "black",
                                        "size": 12},
                                "standoff": 0,},
                        "showgrid": False,
                        "showline": True,
                        "linecolor": "black",
                        "tickfont": {"size": 10}
                        },
                "plot_bgcolor": "white",
                "bargap": 0, "bargroupgap": 0,
                }
        temp_plot = temp_plot.update_layout(layout)

        return temp_plot


@def_function.log
def output_transcript_gene_plot(result_counts, file_path, tab_level=0):
    result_counts = result_counts
    file_path = output_transcript_gene_plot_path

    plot_info_df = pandas.DataFrame()
    for cell_line_compare, plot_info_dict in result_counts.items():
        for gene_transcript, value_dict in plot_info_dict.items():
            [cell_line_1,cell_line_2] = cell_line_compare.split('@')
            [gene_biotype,transcript_biotype] = gene_transcript.split('/')
            temp_df = pandas.DataFrame([[cell_line_1, cell_line_2, gene_biotype, transcript_biotype]+[-1 for i in range(0,plot_x_maxnum+1)]],
                                        columns=["cell_line_1", "cell_line_2", "gene_biotype", "transcript_biotype"]+list(range(0, plot_x_maxnum+1)))
            for num, counts in value_dict.items():
                temp_df[num] = counts
            plot_info_df = pandas.concat([plot_info_df, temp_df], axis=0, ignore_index=True)

    for i in plot_info_df.index:
        cell_line_1 = plot_info_df.at[i, "cell_line_1"]
        cell_line_2 = plot_info_df.at[i, "cell_line_2"]
        gene_biotype = plot_info_df.at[i, "gene_biotype"]
        transcript_biotype = plot_info_df.at[i, "transcript_biotype"]
        plot = make_transcript_gene_plot(df=plot_info_df,
                                         cell_line_1=cell_line_1,
                                         cell_line_2=cell_line_2,
                                         gene_biotype=gene_biotype,
                                         transcript_biotype=transcript_biotype,
                                         tab_level=tab_level+1)
        
        num = 0
        if len(transcript_biotype.split('&')) >= 5:
            num += 1
            print("{}[Result]filename should be: 0004_{}_{}_{}_{}.svg".format('\t'*(tab_level+1),
                                                                              cell_line_1, cell_line_2,
                                                                              gene_biotype, transcript_biotype))
            print("{}[Result]filename is: 0004_{}_{}_{}_{}.svg".format('\t'*(tab_level+1),
                                                                              cell_line_1, cell_line_2,
                                                                              gene_biotype, num))
            plot.write_image("{}0004_{}_{}_{}_{}.svg".format(file_path,
                                                             cell_line_1, cell_line_2,
                                                             gene_biotype, num))
        else:
            plot.write_image("{}0004_{}_{}_{}_{}.svg".format(file_path,
                                                             cell_line_1, cell_line_2,
                                                             gene_biotype, transcript_biotype))
    
    return None


In [17]:
if __name__ == "__main__":
    # load data
    sample_info = def_function.load_sample_info(filename=input_sample_info_filename, tab_level=0)
    total_info = pandas.read_csv(input_df_filename, sep='\t', index_col=0)
    isoform_info_df = pandas.read_csv(input_TSS_TES_filename, sep='\t')
    # 根据ATSS_APA_cutoff对ATSS-APA型gene进行过滤
    readyToDelIndex = isoform_info_df.query("gene_classified=='ATSS-APA' & transcript_counts<@ATSS_APA_cutoff").index
    isoform_info_df = isoform_info_df.drop(index=readyToDelIndex).copy()
    
    # output check plot to confirm the relationship of length and counts
    output_length_counts_check_plot(sample_info=sample_info,
                                    total_info=total_info,
                                    output_filename=output_length_counts_plot_filename,
                                    tab_level=0)


    cell_line_sample_dict = def_function.get_cell_line_sample_dict(sample_info=sample_info,
                                                                   GEO_index=False,
                                                                   tab_level=0)

    # compute expression for each transcript
    total_info = compute_expression_for_each_cell_line(total_info=total_info,
                                                       sample_info=sample_info,
                                                       cell_line_sample_dict=cell_line_sample_dict,
                                                       tab_level=0)


    # compare expression of two cell line
    # and filter transcripts according to log2FC
    transcript_biotype_remove_unclassified = list(set(total_info["transcript_biotype"]))
    transcript_biotype_remove_unclassified.remove("un_classified")
    cell_line_list = list(cell_line_sample_dict.keys())

    result_df = {}  # {"<cell_list_1>/<cell_list_2>_<gene_biotype>/<transcript_biotype>": <check_df>, ...}
    result_counts = {}  # {"<cell_line_1>/<cell_line_2>": {"<gene_biotype>/<transcript_biotype>": {0: <num>,
                        #                                                                          1: <num>, ...},
                        #                                  ...},
                        #  ...}
    for cell_line_1 in range(0,len(cell_line_list)-1):
        for cell_line_2 in range(cell_line_1+1, len(cell_line_list)):
            cell_line_1 = cell_line_list[cell_line_1]
            cell_line_2 = cell_line_list[cell_line_2]
            result_counts["{}@{}".format(cell_line_1, cell_line_2)] = {}
            for gene_biotype in ["all", "protein_coding", "non_protein_coding", "un_classified"]:
                for transcript_biotype in [["all"], transcript_biotype_remove_unclassified]:
                    temp_result = get_counts_of_transcript_of_gene(isoform_info_df=isoform_info_df, total_info=total_info,
                                                                   cell_line_1=cell_line_1, cell_line_2=cell_line_2,
                                                                   gene_biotype=gene_biotype, transcript_biotype=transcript_biotype,
                                                                   log2FC_cutoff=log2FC_cutoff, max_num=plot_x_maxnum,
                                                                   tab_level=0)
                    result_df["{}@{}@{}@{}".format(cell_line_1, cell_line_2, gene_biotype, transcript_biotype)] = temp_result["check_df"]
                    result_counts["{}@{}".format(cell_line_1, cell_line_2)]["{}/{}".format(gene_biotype, '&'.join(transcript_biotype))] = temp_result["result"]

    # output file
    output_check_df(df_dict=result_df, filename=output_check_df_filename)
    output_transcript_gene_plot(result_counts=result_counts, file_path=output_transcript_gene_plot_path)
    total_info.to_csv(output_relative_expression_filename, sep='\t', index=None)

    print("[{}]All blocks finished.".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))))



[Function]load_sample_info start.
	[Time]2023-08-05 20:40:08
	[Paraments]filename: F:/OneDrive/Master/Project/trans/data/0000_sample_info.tsv
	[Paraments]tab_level: 0
[2023-08-05 20:40:08]load_sample_info finished.


[Function]output_length_counts_check_plot start.
	[Time]2023-08-05 20:40:08
	[Paraments]sample_info: <...>
	[Paraments]total_info: <...>
	[Paraments]output_filename: F:/OneDrive/Master/Project/trans/data/other/0004_length_counts.png
	[Paraments]tab_level: 0
[2023-08-05 20:40:24]output_length_counts_check_plot finished.


[Function]get_cell_line_sample_dict start.
	[Time]2023-08-05 20:40:24
	[Paraments]sample_info: <...>
	[Paraments]GEO_index: False
	[Paraments]tab_level: 0
	[Result]cell_line_sample_dict: {'A673': ['GSM6783527'], 'Caco-2': ['GSM6782551']}
[2023-08-05 20:40:24]get_cell_line_sample_dict finished.


[Function]compute_expression_for_each_cell_line start.
	[Time]2023-08-05 20:40:24
	[Paraments]total_info: <...>
	[Paraments]sample_info: <...>
	[Paraments]cell_l

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14036\1927043246.py:256: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14036\1927043246.py:257: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14036\1927043246.py:258: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

	[Result]the counts of gene showing significant expression in A673 and Caco-2 about ATSS-APA and all gene and ['all'] transcript: 8650
	[Result]the counts of transcript showing significant expression in A673 and Caco-2 about ATSS-APA and all gene and ['all'] transcript: 66017
	[Function]compute_counts_of_gene start.
		[Time]2023-08-05 20:40:34
		[Paraments]max_num: 5
		[Paraments]tab_level: 1
	[2023-08-05 20:40:34]compute_counts_of_gene finished.
[2023-08-05 20:40:34]get_counts_of_transcript_of_gene finished.


[Function]get_counts_of_transcript_of_gene start.
	[Time]2023-08-05 20:40:34
	[Paraments]isoform_info_df: <...>
	[Paraments]total_info: <...>
	[Paraments]cell_line_1: A673
	[Paraments]cell_line_2: Caco-2
	[Paraments]gene_biotype: all
	[Paraments]transcript_biotype: ['snRNA', 'TEC', 'transcribed_unitary_pseudogene', 'transcribed_unprocessed_pseudogene', 'scaRNA', 'snoRNA', 'misc_RNA', 'lncRNA', 'processed_transcript', 'nonsense_mediated_decay', 'protein_coding_CDS_not_defined', '

---

In [ ]:
"""
1.对每一个样本中的transcript的表达量进行标准化
  2.1不确定原始数据的rep是如何得到的, 不知道应该用什么计算方法
  2.2姑且认为rep值已经对基因长度进行了校正, 因为rep值将近80%都<=10, 值很低
  2.3所以认为 一个transcript在一个样本中的相对表达量 = 该transcript的rep值 / 该样本所有transcript的rep值的和
  2.4再计算该transcript在相同疾病类型中的不同样本中的平均表达量= 该transcript在一种类型疾病中所有样本的相对表达量的加和 / 该类型疾病的样本数
  2.4再了解一下rep值与transcript长度是否相关
2.筛选得到在指定的两种疾病类型的样本中都存在表达ATSS-APA基因
  2.1根据所有样本的数据筛选ATSS-APA基因id
  2.2获取每种疾病中存在表达的基因id, 以list形式存储
  2.3分别筛选出在单一一种疾病类型中表达的基因，然后将这些基因合并后去重
3.
"""

---

调试

调试

---